In [1]:
import re,string,requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def retrieve_docs_and_clean():
     url = 'https://bola.kompas.com/'
     rqst = requests.get(url)
    
    soup = BeautifulSoup(rqst.content, 'html.parser')

    link = []
    
    for i in soup.find('div', {'class':'most__wrap'}).find_all('a'):
        i['href'] = i['href'] + '?page=all'
        link.append(i['href'])
    
    print(f'Number of links is ({len(link)})')
    print(f'second link is \n')
    print(link[1])


    print('===============================')
    
    
    documents = []
    for i in link:
        r = requests.get(i)
        soup = BeautifulSoup(r.content, 'html.parser')

        sen = []
        for i in soup.find('div', {'class':'read__content'}).find_all('p'):
            sen.append(i.text)
        print(f'number of sentences is {len(sen)} and first sentence is ({sen[0]})')
        documents.append(' '.join(sen))
   
   
    print('===============================')
   
   
    doc_clean = []
    for d in documents:
        doc_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
        doc_test = re.sub(r'@\w+', '', document_test)
        doc_test = document_test.lower()
        doc_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
        doc_test = re.sub(r'[0-9]', '', document_test)
        doc_test = re.sub(r'\s{2,}', ' ', document_test)
        doc_clean.append(document_test)

    
    print(doc_clean)
    
    return documents_clean


def get_similar_articles(q, df):
    print("query:", q)
    print("Article with the highest cosine similarity value: ")
    q = [q]
    q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    print(f'QVec shape is ({q_vec.shape})')
    sim = {}
    for i in range(10):
        sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
    print(sim)
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)

    for k, v in sim_sorted:
        if v != 0.0:
            print("Similarity Value:", v)
            print(docs[k])
            print()


In [4]:
docs = retrieve_docs_and_clean()


Number of links is (10)
second link is 

https://www.kompas.com/sports/read/2023/11/28/21055618/hasil-piala-dunia-u17-2023-kalahkan-mali-perancis-tantang-jerman-di-final?page=all
number of sentences is 18 and first sentence is (KOMPAS.com - Ketua Umum PSSI, Erick Thohir, mengatakan bahwa Piala Dunia U17 2023 menjadi sebuah kesempatan bagi Indonesia untuk membangun dasar kuat sepak bola.)
number of sentences is 27 and first sentence is ()
number of sentences is 22 and first sentence is ()
number of sentences is 22 and first sentence is (KOMPAS.com – Irfan Bachdim menjalani debut yang manis bersama tim barunya, Persik Kediri pada laga pekan ke-20 Liga 1 2023-2024.)
number of sentences is 33 and first sentence is ()
number of sentences is 24 and first sentence is ()
number of sentences is 25 and first sentence is ()
number of sentences is 17 and first sentence is (KOMPAS.com - Argentina akan bersua Jerman pada babak semifinal Piala Dunia U17 2023. Link live streaming Argentina vs Jerman t

In [5]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)
X.shape

(10, 1263)

In [7]:
vect = vectorizer.get_feature_names_out()
df = pd.DataFrame(X.T.toarray(), index=vect)
print(df.shape)
df.head(20)

(1263, 10)


,0,1,2,3,4,5,6,7,8,9
ac,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.104061,0.000000,0.000000,0.032479
acuna,0.000000,0.000000,0.000000,0.000000,0.060127,0.000000,0.000000,0.000000,0.000000,0.000000
ada,0.000000,0.024916,0.000000,0.000000,0.000000,0.058019,0.000000,0.000000,0.000000,0.028415
adalah,0.078154,0.014872,0.023471,0.000000,0.000000,0.017315,0.018113,0.023617,0.020177,0.050880
adapun,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040804,0.000000,0.000000,0.000000
adidas,0.000000,0.000000,0.000000,0.000000,0.000000,0.078011,0.000000,0.000000,0.000000,0.000000
adu,0.000000,0.024916,0.000000,0.000000,0.111795,0.000000,0.000000,0.000000,0.033805,0.000000
afrika,0.000000,0.000000,0.044947,0.000000,0.000000,0.000000,0.000000,0.000000,0.231839,0.000000
agar,0.032735,0.000000,0.000000,0.000000,0.000000,0.000000,0.030347,0.000000,0.033805,0.000000
agresif,0.000000,0.033501,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
df.tail(20)

,0,1,2,3,4,5,6,7,8,9
villalba,0.000000,0.000000,0.000000,0.000000,0.090190,0.000000,0.000000,0.000000,0.000000,0.000000
von,0.000000,0.000000,0.000000,0.000000,0.060127,0.000000,0.000000,0.000000,0.000000,0.000000
vs,0.042910,0.000000,0.103094,0.033451,0.014655,0.000000,0.059671,0.181537,0.088627,0.000000
wakil,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045454,0.000000
waktu,0.000000,0.019894,0.000000,0.061124,0.035705,0.023162,0.000000,0.000000,0.026991,0.000000
waktunya,0.000000,0.000000,0.000000,0.000000,0.000000,0.039005,0.000000,0.000000,0.000000,0.000000
walau,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045454,0.000000
wasit,0.000000,0.067002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
waspada,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081608,0.000000,0.000000,0.000000
wayan,0.000000,0.000000,0.000000,0.068623,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
q = 'windy'
get_similar_articles(q, df)


query: windy
Article with the highest cosine similarity value: 
QVec shape is ((1263,))
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}


In [10]:
q = 'wang'
get_similar_articles(q, df)

query: wang
Article with the highest cosine similarity value: 
QVec shape is ((1263,))
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}
